# 1: 位置 $x$ と時刻 $t$ における波の振幅 $a(x, t)$ の式

$$
a(x, t) = A \sin(kx - \omega t + \phi) \hspace{30pt} (1) 
$$

ここで $A$ は振幅、$k$ は波数、$\omega$ は振動数、$\phi$ は位相を表します。

以下のコードでは波数、振動数、位相を変化させたときの波の形をアニメーションで描画します。

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from matplotlib import cm
from IPython.display import HTML
import matplotlib

# アニメーションの埋め込み制限を増やす
matplotlib.rcParams['animation.embed_limit'] = 100  # MB

# 時空間での波の伝播可視化
# Y軸を時間軸として使用

# ハードコードされたパラメータ
A = 1.0                    # 振幅
k = 1.0                    # 波数
omega = 2.0                # 角周波数
phi = 0.0                  # 初期位相

# 導出される値
wavelength = 2 * np.pi / k
period = 2 * np.pi / omega
phase_velocity = omega / k

print(f"Wave parameters:")
print(f"  Wave number: k = {k:.2f} rad/m")
print(f"  Angular frequency: ω = {omega:.2f} rad/s")
print(f"  Wavelength: λ = {wavelength:.2f} m")
print(f"  Period: T = {period:.2f} s")
print(f"  Phase velocity: c = {phase_velocity:.2f} m/s")

# 空間と時間のグリッド
x = np.linspace(-5, 10, 200)
t = np.linspace(0, 10, 200)
X, T = np.meshgrid(x, t)

# 波動場の計算
# a(x,t) = A * sin(kx - ωt + φ)
Z = A * np.sin(k*X - omega*T + phi)

# 監視する2点
x1, x2 = 0, 2
idx1 = np.argmin(np.abs(x - x1))
idx2 = np.argmin(np.abs(x - x2))

# アニメーション版
fig_anim = plt.figure(figsize=(16, 10))

# アニメーション用のプロット
ax_contour = fig_anim.add_subplot(1, 2, 1)
ax_time = fig_anim.add_subplot(1, 2, 2)

# 固定された時間範囲
t_start = 0
t_end = 10
dt = 0.05
n_frames = int(t_end / dt)

# 固定されたグリッド（アニメーション全体で共通）
t_fixed = np.linspace(t_start, t_end, 200)
x_fixed = np.linspace(-5, 10, 200)
X_fixed, T_fixed = np.meshgrid(x_fixed, t_fixed)
Z_fixed = A * np.sin(k*X_fixed - omega*T_fixed + phi)

def init_anim():
    ax_contour.clear()
    ax_time.clear()
    return []

def animate(frame):
    ax_contour.clear()
    ax_time.clear()
    
    current_time = frame * dt
    
    # カラーコントアプロット（固定されたデータを使用）
    contour = ax_contour.contourf(X_fixed, T_fixed, Z_fixed, 
                                  levels=np.linspace(-1, 1, 21),
                                  cmap=cm.coolwarm, extend='both')
    
    # 等高線を追加
    ax_contour.contour(X_fixed, T_fixed, Z_fixed, 
                       levels=np.linspace(-1, 1, 11),
                       colors='black', alpha=0.3, linewidths=0.5)
    
    # 現在時刻を示す線（これだけが動く）
    ax_contour.axhline(y=current_time, color='green', linewidth=3, linestyle='-', label='Current time')
    
    # 監視点の位置を示す線
    ax_contour.axvline(x=x1, color='red', linewidth=1, linestyle=':', alpha=0.7)
    ax_contour.axvline(x=x2, color='blue', linewidth=1, linestyle=':', alpha=0.7)
    
    # 監視点での軌跡を強調
    ax_contour.plot(np.full_like(t_fixed, x1), t_fixed, 'w-', linewidth=2, alpha=0.5)
    ax_contour.plot(np.full_like(t_fixed, x2), t_fixed, 'w-', linewidth=2, alpha=0.5)
    
    ax_contour.set_xlabel('Position x [m]', fontsize=12)
    ax_contour.set_ylabel('Time t [s]', fontsize=12)
    ax_contour.set_title(f'Space-Time Wave Field (Contour Plot)', fontsize=14)
    ax_contour.set_xlim(-5, 10)
    ax_contour.set_ylim(t_start, t_end)
    
    # カラーバーを追加（初回のみ）
    if frame == 0:
        cbar = plt.colorbar(contour, ax=ax_contour)
        cbar.set_label('Amplitude', fontsize=12)
    
    # 右側のパネル：現在時刻での空間分布
    current_wave = A * np.sin(k*x - omega*current_time + phi)
    ax_time.plot(x, current_wave, 'k-', linewidth=2)
    
    # 監視点をマーク
    ax_time.plot(x1, A * np.sin(k*x1 - omega*current_time + phi), 'ro', markersize=10, label=f'x = {x1}')
    ax_time.plot(x2, A * np.sin(k*x2 - omega*current_time + phi), 'bo', markersize=10, label=f'x = {x2}')
    
    ax_time.set_xlabel('Position x [m]', fontsize=12)
    ax_time.set_ylabel('Amplitude', fontsize=12)
    ax_time.set_title(f'Spatial Distribution at t = {current_time:.2f} s', fontsize=14)
    ax_time.set_xlim(-5, 10)
    ax_time.set_ylim(-1.5, 1.5)
    ax_time.grid(True, alpha=0.3)
    ax_time.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax_time.axvline(x=x1, color='red', linestyle=':', alpha=0.5)
    ax_time.axvline(x=x2, color='blue', linestyle=':', alpha=0.5)
    ax_time.legend()
    
    # 波の情報を表示
    phase_at_x1 = (k*x1 - omega*current_time + phi) % (2*np.pi)
    phase_at_x2 = (k*x2 - omega*current_time + phi) % (2*np.pi)
    info_text = f'Phase at x={x1}: {phase_at_x1:.2f} rad\nPhase at x={x2}: {phase_at_x2:.2f} rad'
    ax_time.text(0.05, 0.95, info_text, transform=ax_time.transAxes, 
                verticalalignment='top', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    return []

# アニメーションの作成
anim = animation.FuncAnimation(fig_anim, animate, init_func=init_anim,
                              frames=n_frames, interval=50,
                              blit=False, repeat=True)

# HTMLとして表示
HTML(anim.to_jshtml())

# 2: F-K変換

F-K変換は、波の振幅を空間と時間の関数として表現する方法です。波の振幅 $a(x, t)$ をフーリエ変換すると、以下のように表されます。

$$
\begin{align*}
A(\omega_i, x) &= \frac{1}{N} \sum_{n=0}^{N-1} a(x, t_n) e^{-i \omega_i t_n} \hspace{30pt} (2-3) \\
A(k_i, \omega_i) &= \frac{1}{M} \sum_{m=0}^{M-1} A(\omega_i, x_m) e^{-i k_i x_m} \hspace{30pt} (2-4)
\end{align*}
$$

以下のコードでは、F-K変換を用いて波の振幅を空間と時間の関数として表現し、アニメーションで表示します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors as colors

# F-K変換の可視化
# (x,t)空間から(k,ω)空間への変換を理解する

# パラメータ設定
# 位相速度の差が明瞭になるように設定
wave_params = [
    {'k': 10.0, 'omega': 3.0, 'A': 1.0, 'phi': 0.0},      # 波1: c = 0.3 m/s (低速)
    {'k': 5.0, 'omega': 5.0, 'A': 0.8, 'phi': np.pi/4},   # 波2: c = 1.0 m/s (中速)
    {'k': 4.0, 'omega': 20.0, 'A': 0.6, 'phi': -np.pi/3}, # 波3: c = 5.0 m/s (高速)
]

# 各波の位相速度を計算
for params in wave_params:
    params['c'] = params['omega'] / params['k']

# 空間と時間のグリッド
nx, nt = 256, 256
x_max, t_max = 10, 10
x = np.linspace(0, x_max, nx)
t = np.linspace(0, t_max, nt)
dx = x[1] - x[0]
dt = t[1] - t[0]

# 波動場の生成（複数の波の重ね合わせ）
X, T = np.meshgrid(x, t)
wave_field = np.zeros_like(X)

print("波のパラメータ:")
for i, params in enumerate(wave_params):
    wave_i = params['A'] * np.sin(params['k']*X - params['omega']*T + params['phi'])
    wave_field += wave_i
    print(f"  波{i+1}: k={params['k']:.1f} rad/m, ω={params['omega']:.1f} rad/s, A={params['A']:.1f}, c={params['c']:.2f} m/s")

# F-K変換の実行
# Step 1: 時間方向のFFT（各位置で）
fft_t = np.fft.fft(wave_field, axis=0)
omega_array = 2 * np.pi * np.fft.fftfreq(nt, dt)

# Step 2: 空間方向のFFT（各周波数で）
fft_xt = np.fft.fft(fft_t, axis=1)
k_array = 2 * np.pi * np.fft.fftfreq(nx, dx)

# F-Kスペクトルの計算（パワースペクトル密度）
# FFTの正規化とPSDへの変換
fk_spectrum = np.abs(fft_xt)**2 / (nx * nt)  # FFTの正規化
fk_spectrum = fk_spectrum * 2 * dt * dx  # 片側スペクトルのためのファクター2を含む

# 正の周波数・波数領域のみ抽出
omega_positive = omega_array[:nt//2]
k_positive = k_array[:nx//2]
fk_spectrum_positive = fk_spectrum[:nt//2, :nx//2]

# 静的な可視化
fig = plt.figure(figsize=(16, 10))

# 断面の位置を定義
x_section = x_max / 4  # x = 2.5 m
omega_section = 5.0    # ω = 5.0 rad/s
x_idx = np.argmin(np.abs(x - x_section))
omega_idx = np.argmin(np.abs(omega_positive - omega_section))

# 1. 時空間波動場（時間軸を下から上に）
ax1 = fig.add_subplot(2, 3, 1)
im1 = ax1.imshow(wave_field, extent=[0, x_max, 0, t_max], 
                 cmap='coolwarm', aspect='auto', vmin=-2, vmax=2, origin='lower')
ax1.set_xlabel('Position x [m]')
ax1.set_ylabel('Time t [s]')
ax1.set_title('Wave Field in (x,t) Space')
plt.colorbar(im1, ax=ax1, label='Amplitude')

# 理論的な波の軌跡を追加
for params in wave_params:
    t_line = np.linspace(0, t_max, 100)
    x_line = params['c'] * t_line
    mask = x_line <= x_max
    ax1.plot(x_line[mask], t_line[mask], '--', linewidth=1, alpha=0.7)

# 断面線を追加（ax4とax5の位置）
ax1.axvline(x=x_section, color='green', linestyle=':', linewidth=2, label=f'x = {x_section:.1f} m')
ax1.legend()

# 2. F-Kスペクトル（2D）
ax2 = fig.add_subplot(2, 3, 2)
# log10スケールでのパワースペクトル密度
log_psd = np.log10(fk_spectrum_positive + 1e-10)
im2 = ax2.imshow(log_psd, 
                 extent=[0, k_positive.max(), 0, omega_positive.max()],
                 cmap='viridis', aspect='auto', origin='lower')
ax2.set_xlabel('Wave number k [rad/m]')
ax2.set_ylabel('Angular frequency ω [rad/s]')
ax2.set_title('F-K Spectrum (log10 PSD)')
ax2.set_xlim(0, 15)
ax2.set_ylim(0, 25)
plt.colorbar(im2, ax=ax2, label='log10(PSD)')

# 理論的な分散関係を追加
k_theory = np.linspace(0, 15, 100)
for params in wave_params:
    omega_theory = params['c'] * k_theory
    ax2.plot(k_theory, omega_theory, 'w--', linewidth=1, alpha=0.7)

# 断面線を追加（ax3の位置）
ax2.axhline(y=omega_section, color='red', linestyle=':', linewidth=2, label=f'ω = {omega_section:.1f} rad/s')
ax2.legend()

# 3. 特定周波数での波数スペクトル
ax3 = fig.add_subplot(2, 3, 3)
k_spectrum_at_omega = fk_spectrum_positive[omega_idx, :]
ax3.plot(k_positive, k_spectrum_at_omega)
ax3.set_xlabel('Wave number k [rad/m]')
ax3.set_ylabel('PSD')
ax3.set_title(f'Wave Number Spectrum at ω = {omega_positive[omega_idx]:.2f} rad/s')
ax3.set_xlim(0, 15)
ax3.grid(True, alpha=0.3)

# ピークの位置をマーク
peaks_idx = np.where(k_spectrum_at_omega > np.max(k_spectrum_at_omega) * 0.1)[0]
for idx in peaks_idx:
    if k_positive[idx] > 0:
        c_est = omega_positive[omega_idx] / k_positive[idx]
        ax3.axvline(x=k_positive[idx], color='red', linestyle=':', alpha=0.5)
        ax3.text(k_positive[idx], np.max(k_spectrum_at_omega)*0.8, 
                f'c≈{c_est:.1f}', ha='center', fontsize=9)

# 4. 時間領域の信号（特定位置）
ax4 = fig.add_subplot(2, 3, 4)
time_signal = wave_field[:, x_idx]
ax4.plot(t, time_signal)
ax4.set_xlabel('Time t [s]')
ax4.set_ylabel('Amplitude')
ax4.set_title(f'Time Signal at x = {x[x_idx]:.1f} m')
ax4.grid(True, alpha=0.3)

# 5. 周波数スペクトル（特定位置、PSD）
ax5 = fig.add_subplot(2, 3, 5)
# パワースペクトル密度の計算
freq_psd = np.abs(fft_t[:nt//2, x_idx])**2 / nt * 2 * dt
ax5.plot(omega_positive, freq_psd)
ax5.set_xlabel('Angular frequency ω [rad/s]')
ax5.set_ylabel('PSD')
ax5.set_title(f'Frequency Spectrum at x = {x[x_idx]:.1f} m')
ax5.set_xlim(0, 25)
ax5.grid(True, alpha=0.3)

# 主要な周波数成分をマーク
peaks_omega = omega_positive[freq_psd > np.max(freq_psd) * 0.2]
for peak in peaks_omega:
    ax5.axvline(x=peak, color='red', linestyle=':', alpha=0.5)

# 6. 位相速度の抽出（マーカーのサイズをlog10(PSD)に比例）
ax6 = fig.add_subplot(2, 3, 6)

# F-Kスペクトルのピークから位相速度を推定
c_values = []
omega_values = []
psd_values = []

# すべての周波数に対してF-Kスペクトルからピークを抽出
for i in range(len(omega_positive)):
    if omega_positive[i] > 1.0:  # 低周波ノイズを除外
        k_peak_idx = np.argmax(fk_spectrum_positive[i, :])
        if k_positive[k_peak_idx] > 0.5:  # 低波数ノイズを除外
            c_estimated = omega_positive[i] / k_positive[k_peak_idx]
            if c_estimated < 10:  # 異常値を除外
                c_values.append(c_estimated)
                omega_values.append(omega_positive[i])
                psd_values.append(fk_spectrum_positive[i, k_peak_idx])

# 与えられた波のパラメータに対応する理論値を追加
theory_c_values = []
theory_omega_values = []
theory_psd_values = []

for params in wave_params:
    # 対応する周波数のインデックスを見つける
    omega_idx_theory = np.argmin(np.abs(omega_positive - params['omega']))
    # 対応する波数のインデックスを見つける
    k_idx_theory = np.argmin(np.abs(k_positive - params['k']))
    
    # 理論値を追加
    theory_omega_values.append(params['omega'])
    theory_c_values.append(params['c'])
    # 実際のF-KスペクトルからPSD値を取得
    theory_psd_values.append(fk_spectrum_positive[omega_idx_theory, k_idx_theory])
    
    # デバッグ情報
    print(f"波{wave_params.index(params)+1}: ω={params['omega']:.1f}, k={params['k']:.1f}, " +
          f"ω_idx={omega_idx_theory}, k_idx={k_idx_theory}, " +
          f"PSD={fk_spectrum_positive[omega_idx_theory, k_idx_theory]:.6f}")

# すべてのPSD値を結合（正規化のため）
all_psd_values = psd_values + theory_psd_values

# log10(PSD)を計算
log_psd_values = np.log10(np.array(psd_values) + 1e-10)
log_theory_psd_values = np.log10(np.array(theory_psd_values) + 1e-10)
all_log_psd_values = np.log10(np.array(all_psd_values) + 1e-10)

# 全体で正規化してサイズを決定（20から300の範囲）
size_min, size_max = 20, 300
log_psd_min = np.min(all_log_psd_values)
log_psd_max = np.max(all_log_psd_values)

# 推定値のマーカーサイズ
if log_psd_max > log_psd_min:
    log_psd_normalized = (log_psd_values - log_psd_min) / (log_psd_max - log_psd_min)
    marker_sizes = size_min + (size_max - size_min) * log_psd_normalized
    
    # 理論値のマーカーサイズ
    theory_log_psd_normalized = (log_theory_psd_values - log_psd_min) / (log_psd_max - log_psd_min)
    theory_marker_sizes = size_min + (size_max - size_min) * theory_log_psd_normalized
else:
    marker_sizes = np.full(len(psd_values), size_min)
    theory_marker_sizes = np.full(len(theory_psd_values), size_min)

# 推定値の散布図（サイズをPSDに対応）
scatter1 = ax6.scatter(omega_values, c_values, s=marker_sizes, 
                      c='blue', alpha=0.6, edgecolors='darkblue', linewidth=0.5, 
                      label='Estimated from F-K')

# 理論値の散布図（強調表示）
scatter2 = ax6.scatter(theory_omega_values, theory_c_values, s=theory_marker_sizes, 
                      c='red', alpha=0.8, edgecolors='darkred', linewidth=2, 
                      marker='s', label='Theoretical values')

ax6.set_xlabel('Angular frequency ω [rad/s]')
ax6.set_ylabel('Phase velocity c [m/s]')
ax6.set_title('Extracted Phase Velocities (size ∝ log10(PSD))')
ax6.set_xlim(0, 25)
ax6.set_ylim(0, 6)
ax6.grid(True, alpha=0.3)

# 理論値の水平線を追加
for i, params in enumerate(wave_params):
    ax6.axhline(y=params['c'], color=f'C{i}', linestyle='--', 
                alpha=0.5, linewidth=1)
    ax6.text(22, params['c'], f"Wave {i+1}: c = {params['c']:.2f} m/s", 
             fontsize=9, va='center')

ax6.legend(loc='upper left')

plt.tight_layout()

# 3: 空間自己相関法（SPAC法）の基礎 - 2地点間の波の相関

空間自己相関法（SPAC法）の基本原理を理解するために、2つの観測点での単純な正弦波の相関を考えます。

1次元の波の式：
$$u(x, t) = A \sin(kx - \omega t + \phi)$$

2つの観測点（$x_1 = 0$と$x_2 = r$）での波形：
- 観測点1：$u_1(t) = A \sin(-\omega t + \phi)$
- 観測点2：$u_2(t) = A \sin(kr - \omega t + \phi)$

これらの積の時間平均を取ると：
$$\langle u_1(t) u_2(t) \rangle = \frac{A^2}{2} \cos(kr)$$

以下のアニメーションでは、ノイズを含む実際の観測を模擬し、時間平均が理論値に収束する様子を可視化します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import matplotlib

# アニメーションの埋め込み制限を増やす
matplotlib.rcParams['animation.embed_limit'] = 100  # MB

# 波のパラメータ
A = 1.0          # 振幅
k = 2.0          # 波数 [rad/m]
omega = 4.0      # 角周波数 [rad/s]
phi = 0.0        # 初期位相

# 観測点の設定
x1 = 0.0         # 観測点1の位置 [m]
r = 3.0          # 観測点間距離 [m]
x2 = x1 + r      # 観測点2の位置 [m]

# ノイズレベル
noise_level = 0.3  # 信号振幅に対するノイズの比率

# 導出される値
wavelength = 2 * np.pi / k
period = 2 * np.pi / omega
phase_velocity = omega / k
phase_diff = k * r  # 位相差 [rad]

print(f"波のパラメータ:")
print(f"  波数: k = {k:.2f} rad/m")
print(f"  角周波数: ω = {omega:.2f} rad/s")
print(f"  波長: λ = {wavelength:.2f} m")
print(f"  周期: T = {period:.2f} s")
print(f"  位相速度: c = {phase_velocity:.2f} m/s")
print(f"\n観測点情報:")
print(f"  観測点1: x₁ = {x1:.1f} m")
print(f"  観測点2: x₂ = {x2:.1f} m")
print(f"  観測点間距離: r = {r:.1f} m")
print(f"  位相差: kr = {phase_diff:.3f} rad = {phase_diff/np.pi:.3f}π")
print(f"  理論相関値: cos(kr) = {np.cos(phase_diff):.3f}")

# 時間設定
dt = 0.02        # 時間刻み [s]
t_max = 20.0     # 最大時間 [s]
t = np.arange(0, t_max, dt)
n_steps = len(t)

# 波形の生成（ノイズなし）
u1_clean = A * np.sin(-omega * t + phi)
u2_clean = A * np.sin(k * r - omega * t + phi)

# ノイズの生成（時間相関のあるノイズ）
np.random.seed(42)  # 再現性のため
from scipy.ndimage import gaussian_filter1d

# ホワイトノイズを生成してガウシアンフィルタで平滑化
noise1_white = np.random.randn(n_steps)
noise2_white = np.random.randn(n_steps)
sigma_time = 5  # 時間相関の強さ
noise1 = gaussian_filter1d(noise1_white, sigma_time) * noise_level * A
noise2 = gaussian_filter1d(noise2_white, sigma_time) * noise_level * A

# ノイズ付き波形
u1 = u1_clean + noise1
u2 = u2_clean + noise2

# 理論値の計算
theoretical_correlation = (A**2 / 2) * np.cos(k * r)
theoretical_correlation_coef = np.cos(k * r)

# アニメーション用の設定
fig = plt.figure(figsize=(16, 12))

# サブプロットの配置
gs = fig.add_gridspec(3, 2, height_ratios=[1.5, 1.5, 1], hspace=0.3)
ax_wave1 = fig.add_subplot(gs[0, 0])
ax_wave2 = fig.add_subplot(gs[0, 1])
ax_product = fig.add_subplot(gs[1, :])
ax_correlation = fig.add_subplot(gs[2, :])

# 初期化関数
def init():
    return []

# アニメーション関数
def animate(frame):
    # 現在時刻
    current_idx = frame * 5  # 5ステップごとに更新
    if current_idx >= n_steps:
        current_idx = n_steps - 1
    
    current_time = t[current_idx]
    
    # クリア
    ax_wave1.clear()
    ax_wave2.clear()
    ax_product.clear()
    ax_correlation.clear()
    
    # 1. 観測点1の波形
    ax_wave1.plot(t[:current_idx], u1[:current_idx], 'b-', linewidth=1.5, label='With noise')
    ax_wave1.plot(t[:current_idx], u1_clean[:current_idx], 'b--', linewidth=1, alpha=0.5, label='Clean signal')
    ax_wave1.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax_wave1.set_xlim(0, t_max)
    ax_wave1.set_ylim(-2*A, 2*A)
    ax_wave1.set_xlabel('Time [s]')
    ax_wave1.set_ylabel('Amplitude')
    ax_wave1.set_title(f'Observation Point 1 (x = {x1:.1f} m)')
    ax_wave1.grid(True, alpha=0.3)
    ax_wave1.legend(loc='upper right')
    
    # 現在値をマーク
    if current_idx > 0:
        ax_wave1.plot(current_time, u1[current_idx-1], 'ro', markersize=8)
    
    # 2. 観測点2の波形
    ax_wave2.plot(t[:current_idx], u2[:current_idx], 'g-', linewidth=1.5, label='With noise')
    ax_wave2.plot(t[:current_idx], u2_clean[:current_idx], 'g--', linewidth=1, alpha=0.5, label='Clean signal')
    ax_wave2.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax_wave2.set_xlim(0, t_max)
    ax_wave2.set_ylim(-2*A, 2*A)
    ax_wave2.set_xlabel('Time [s]')
    ax_wave2.set_ylabel('Amplitude')
    ax_wave2.set_title(f'Observation Point 2 (x = {x2:.1f} m)')
    ax_wave2.grid(True, alpha=0.3)
    ax_wave2.legend(loc='upper right')
    
    # 現在値をマーク
    if current_idx > 0:
        ax_wave2.plot(current_time, u2[current_idx-1], 'ro', markersize=8)
    
    # 3. 瞬時積（瞬時相関）
    product = u1[:current_idx] * u2[:current_idx]
    product_clean = u1_clean[:current_idx] * u2_clean[:current_idx]
    
    ax_product.plot(t[:current_idx], product, 'k-', linewidth=1.5, label='u₁(t) × u₂(t) with noise')
    ax_product.plot(t[:current_idx], product_clean, 'k--', linewidth=1, alpha=0.5, label='u₁(t) × u₂(t) clean')
    ax_product.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax_product.axhline(y=theoretical_correlation, color='r', linestyle='--', 
                       linewidth=2, alpha=0.7, label=f'Theory: {theoretical_correlation:.3f}')
    ax_product.set_xlim(0, t_max)
    ax_product.set_ylim(-A**2, A**2)
    ax_product.set_xlabel('Time [s]')
    ax_product.set_ylabel('Product')
    ax_product.set_title('Instantaneous Product u₁(t) × u₂(t)')
    ax_product.grid(True, alpha=0.3)
    ax_product.legend(loc='upper right')
    
    # 4. 時間平均（累積平均）と相関係数
    if current_idx > 1:
        # 累積平均の計算
        cumsum_product = np.cumsum(product)
        cumsum_u1_squared = np.cumsum(u1[:current_idx]**2)
        cumsum_u2_squared = np.cumsum(u2[:current_idx]**2)
        
        time_indices = np.arange(1, current_idx + 1)
        mean_product = cumsum_product / time_indices
        mean_u1_squared = cumsum_u1_squared / time_indices
        mean_u2_squared = cumsum_u2_squared / time_indices
        
        # 相関係数の計算（分母がゼロでないことを確認）
        denominator = np.sqrt(mean_u1_squared * mean_u2_squared)
        correlation_coef = np.divide(mean_product, denominator, 
                                   out=np.zeros_like(mean_product), 
                                   where=denominator!=0)
        
        # 相関係数をプロット
        ax_correlation.plot(t[:current_idx], correlation_coef, 'purple', linewidth=2, 
                           label='Correlation coefficient')
        
        # 現在の相関係数値を表示
        current_corr = correlation_coef[-1]
        ax_correlation.text(0.02, 0.95, f'Current ρ = {current_corr:.3f}', 
                           transform=ax_correlation.transAxes, fontsize=12,
                           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
                           verticalalignment='top')
    
    ax_correlation.axhline(y=theoretical_correlation_coef, color='r', linestyle='--', 
                          linewidth=2, alpha=0.7, label=f'Theory: cos(kr) = {theoretical_correlation_coef:.3f}')
    ax_correlation.set_xlim(0, t_max)
    ax_correlation.set_ylim(-1.2, 1.2)
    ax_correlation.set_xlabel('Time [s]')
    ax_correlation.set_ylabel('Correlation Coefficient')
    ax_correlation.set_title('Time-averaged Correlation Coefficient')
    ax_correlation.grid(True, alpha=0.3)
    ax_correlation.legend(loc='upper right')
    
    # 全体のタイトル
    fig.suptitle(f'Spatial Autocorrelation: Two-point Wave Correlation (t = {current_time:.2f} s)', 
                 fontsize=16)
    
    return []

# アニメーションの作成
n_frames = n_steps // 5  # 5ステップごとにフレーム
anim = animation.FuncAnimation(fig, animate, init_func=init,
                              frames=n_frames, interval=50,
                              blit=False, repeat=True)

plt.tight_layout()
HTML(anim.to_jshtml())

## 観測点間距離と相関の関係

次に、観測点間距離 $r$ を変化させたときの相関係数の変化を可視化します。
理論的には、相関係数は $\cos(kr) = \cos(2\pi r/\lambda)$ となり、これは第1種0次ベッセル関数 $J_0(kr)$ の特殊な場合（1方向からの波）です。

観測点間距離が：
- $r = 0$（同じ場所）：相関係数 = 1（完全に一致）
- $r = \lambda/4$：相関係数 = 0（位相が90°ずれ）
- $r = \lambda/2$：相関係数 = -1（逆位相）
- $r = \lambda$：相関係数 = 1（再び一致）

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from scipy.special import j0
import matplotlib

# アニメーションの埋め込み制限を増やす
matplotlib.rcParams['animation.embed_limit'] = 100  # MB

# 波のパラメータ
A = 1.0          # 振幅
k = 2 * np.pi    # 波数 [rad/m]（波長1mに設定）
omega = 4 * np.pi # 角周波数 [rad/s]
phi = 0.0        # 初期位相

# 導出される値
wavelength = 2 * np.pi / k  # = 1.0 m
period = 2 * np.pi / omega
phase_velocity = omega / k

print(f"波のパラメータ:")
print(f"  波長: λ = {wavelength:.2f} m")
print(f"  周期: T = {period:.2f} s")
print(f"  位相速度: c = {phase_velocity:.2f} m/s")

# 観測点間距離の範囲
r_max = 2.0 * wavelength  # 最大距離は波長の2倍
r_values = np.linspace(0, r_max, 200)

# 理論的な相関係数
correlation_theory = np.cos(k * r_values)

# ベッセル関数（等方的波動場の場合）
bessel_values = j0(k * r_values)

# 時間設定（短時間のシミュレーション）
dt = 0.01
t_sim = 10.0  # シミュレーション時間
t = np.arange(0, t_sim, dt)
n_steps = len(t)

# ノイズレベル
noise_level = 0.2

# アニメーション用の図
fig = plt.figure(figsize=(16, 10))

# サブプロットの配置
gs = fig.add_gridspec(2, 2, width_ratios=[1.5, 1], height_ratios=[1, 1], hspace=0.3, wspace=0.3)
ax_main = fig.add_subplot(gs[:, 0])
ax_wave1 = fig.add_subplot(gs[0, 1])
ax_wave2 = fig.add_subplot(gs[1, 1])

# アニメーション設定
n_frames = 100
r_animation = np.linspace(0, r_max, n_frames)

def init():
    return []

def animate(frame):
    # 現在の観測点間距離
    r_current = r_animation[frame]
    
    # クリア
    ax_main.clear()
    ax_wave1.clear()
    ax_wave2.clear()
    
    # 1. メインプロット：相関係数 vs 観測点間距離
    ax_main.plot(r_values, correlation_theory, 'b-', linewidth=2, label='cos(kr): 1D wave')
    ax_main.plot(r_values, bessel_values, 'g--', linewidth=2, label='J₀(kr): Isotropic field')
    ax_main.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax_main.axhline(y=1, color='k', linestyle=':', alpha=0.3)
    ax_main.axhline(y=-1, color='k', linestyle=':', alpha=0.3)
    
    # 現在の位置をマーク
    current_corr = np.cos(k * r_current)
    current_bessel = j0(k * r_current)
    ax_main.plot(r_current, current_corr, 'ro', markersize=12, label=f'Current: r = {r_current:.2f} m')
    ax_main.plot(r_current, current_bessel, 'go', markersize=10)
    
    # 波長の倍数をマーク
    for i in range(int(r_max/wavelength) + 1):
        ax_main.axvline(x=i*wavelength, color='gray', linestyle=':', alpha=0.5)
        if i > 0:
            ax_main.text(i*wavelength, -1.3, f'{i}λ', ha='center', fontsize=10)
    
    # 波長の1/4倍数もマーク（薄い線で）
    for i in range(int(4*r_max/wavelength) + 1):
        if i % 4 != 0:  # 整数倍以外
            ax_main.axvline(x=i*wavelength/4, color='gray', linestyle=':', alpha=0.2)
    
    ax_main.set_xlim(0, r_max)
    ax_main.set_ylim(-1.5, 1.5)
    ax_main.set_xlabel('Station Separation r [m]', fontsize=12)
    ax_main.set_ylabel('Correlation Coefficient', fontsize=12)
    ax_main.set_title('Spatial Autocorrelation vs Station Separation', fontsize=14)
    ax_main.grid(True, alpha=0.3)
    ax_main.legend(loc='upper right')
    
    # 現在の相関値を表示
    ax_main.text(0.02, 0.95, f'r = {r_current:.2f} m = {r_current/wavelength:.2f}λ\n' + 
                            f'cos(kr) = {current_corr:.3f}\n' +
                            f'J₀(kr) = {current_bessel:.3f}',
                transform=ax_main.transAxes, fontsize=12,
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
                verticalalignment='top')
    
    # 2. 観測点1の波形（固定位置 x=0）
    t_display = np.linspace(0, 2, 200)  # 2秒分表示
    u1 = A * np.sin(-omega * t_display + phi)
    
    # ノイズを追加
    np.random.seed(42)
    noise1 = np.random.randn(len(t_display)) * noise_level * A
    u1_noisy = u1 + noise1
    
    ax_wave1.plot(t_display, u1, 'b-', linewidth=2, label='Clean')
    ax_wave1.plot(t_display, u1_noisy, 'b-', linewidth=1, alpha=0.5, label='With noise')
    ax_wave1.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax_wave1.set_xlim(0, 2)
    ax_wave1.set_ylim(-1.5*A, 1.5*A)
    ax_wave1.set_xlabel('Time [s]')
    ax_wave1.set_ylabel('Amplitude')
    ax_wave1.set_title('Station 1 (x = 0)')
    ax_wave1.grid(True, alpha=0.3)
    ax_wave1.legend()
    
    # 3. 観測点2の波形（可変位置 x=r）
    u2 = A * np.sin(k * r_current - omega * t_display + phi)
    
    # ノイズを追加（異なるシード）
    np.random.seed(43)
    noise2 = np.random.randn(len(t_display)) * noise_level * A
    u2_noisy = u2 + noise2
    
    ax_wave2.plot(t_display, u2, 'g-', linewidth=2, label='Clean')
    ax_wave2.plot(t_display, u2_noisy, 'g-', linewidth=1, alpha=0.5, label='With noise')
    ax_wave2.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax_wave2.set_xlim(0, 2)
    ax_wave2.set_ylim(-1.5*A, 1.5*A)
    ax_wave2.set_xlabel('Time [s]')
    ax_wave2.set_ylabel('Amplitude')
    ax_wave2.set_title(f'Station 2 (x = {r_current:.2f} m)')
    ax_wave2.grid(True, alpha=0.3)
    ax_wave2.legend()
    
    # 位相差を色で表示
    phase_diff = (k * r_current) % (2 * np.pi)
    phase_diff_deg = phase_diff * 180 / np.pi
    
    # 波形の類似度を背景色で表現
    if abs(current_corr) > 0.8:
        bg_color = 'lightgreen'
        similarity = 'High correlation'
    elif abs(current_corr) < 0.2:
        bg_color = 'lightgray'
        similarity = 'Low correlation'
    else:
        bg_color = 'lightyellow'
        similarity = 'Medium correlation'
    
    ax_wave2.text(0.5, 0.02, f'Phase difference: {phase_diff_deg:.1f}°\n{similarity}', 
                 transform=ax_wave2.transAxes, fontsize=10,
                 bbox=dict(boxstyle='round', facecolor=bg_color, alpha=0.7),
                 ha='center')
    
    # 全体のタイトル
    fig.suptitle(f'Effect of Station Separation on Spatial Correlation', fontsize=16)
    
    return []

# アニメーションの作成
anim = animation.FuncAnimation(fig, animate, init_func=init,
                              frames=n_frames, interval=100,
                              blit=False, repeat=True)

plt.tight_layout()
HTML(anim.to_jshtml())